# Setting up a mock DB to store Songs, users, and playlists

![](db_tables_illu.png)

In [1]:
import sys
sys.path.insert(0, '../')
from spotify_users import User

In [2]:
user1 = User()
user1.user_top_50('')



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=933b2272683d4796a86dd9a524514edb&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8888%2Flab&scope=playlist-modify-public+user-library-read+user-top-read in your browser




Enter the URL you were redirected to:  http://localhost:8888/lab/workspaces/auto-8?code=AQDxy6hnRvKcCdPn9RmTxfv1EgAnAJg4put8wPsgM9vZBKdRb5sRrZof12XGjSe5mrvPEbVdPGd1JL6hl0gH9WFLx9P8jaHqCh8QaZ_qh9hjDMr3nYlizCf7p8BIQDQ6MkgouauLdcXa9Bt0QrPVJ5VlNO83_jEzF4abFUy03_hjMTXQmX-0o6x2tPQC-CKJuByfCKJ4MZLVtWFV5zIqO3BBrBbBph04vzUiMDRMdCRoL5PYM99w


{'Top Tracks tokens': {'access_token': 'BQD7s82yoN75zDFAJDJUhdQp1Q8TkizFX6-j5ZQKeZsBaiMKltlSPMcYlPjSEs4e6e6xrl-4-VF1HQvtUxjmA9CX--lZww5LggyF09mc-9R0NY21_XBgeHTHq8276JGcuTMbcr2Jm4KPzXfc2uKL7uvVIhp_s440BWuAk0NmjJ5Nx7ICyXLO2wqGmFTa8-rvnq1N5kkYvfikwLRqTNA',
  'token_type': 'Bearer',
  'expires_in': 3600,
  'refresh_token': 'AQAw1qiYzEo5YictluvpDw0MttffoGGafrHjGAbQd2z6HJ-MsRs55pquuWREItFy_RMx54-GtSgWj82vuoFhFKTmbWDZlg-8a6PNNk7K-AApt2n5DTPeCxnzYqY16g51SjQ',
  'scope': 'playlist-modify-public user-library-read user-top-read',
  'expires_at': 1599775598},
 'User': {''},
 'Top Track IDs': ['0akyEssGRVHstqCSWXusJL',
  '02gaYAEdeR6poHcBH1KUQF',
  '0NeJjNlprGfZpeX2LQuN6c',
  '6plO0gM4tUvRC9TKFGIuaN',
  '54KsfVVnN4YWI2mMrnyUcC',
  '5iSpfk6cDOSYePagAoG639',
  '6KseaEAFSS63N2NPZtDnRL',
  '1jecO8NeYLsVWVptITz4c1',
  '1s3WD4gbNoEXHiuSTmAKaK',
  '57mLRN6tfXwTRvp9oPWpop',
  '52N0IV8hLVkRmnpFclmCzK',
  '4zFPUEMucYleIIUnYVoeZf',
  '1IF5UcqRO42D12vYwceOY6',
  '4cJOLN346rtOty3UPACsao',
  '7kWFRZdedr2gtfE8JDu